In [1]:
#imports
import pandas as pd
from tabula import read_pdf
import os
#settings
pd.set_option('display.max_columns', 0) # 0 = autodetect
pd.set_option('display.max_rows', 6000)

In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Define the path to the "Data" folder in the parent directory
data_folder = os.path.join(parent_directory, "Data")
raw_data_path = os.path.join(data_folder, "bee-data-RAW.csv")

In [3]:
raw_df = pd.read_csv(raw_data_path)

In [4]:
raw_df.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,County ANSI,Zip Code,Region,watershed_code,Watershed,Commodity,Domain,Domain Category,"HONEY, BEE COLONIES - ADDED & REPLACED, MEASURED IN COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES - ADDED & REPLACED, MEASURED IN COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES - INVENTORY, MAX, MEASURED IN COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES - INVENTORY, MAX, MEASURED IN COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DISORDER, MEASURED IN COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DISORDER, MEASURED IN COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, AFFECTED BY DISEASE - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, AFFECTED BY DISEASE - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, AFFECTED BY OTHER CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, AFFECTED BY OTHER CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, AFFECTED BY PESTS ((EXCL VARROA MITES)) - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, AFFECTED BY PESTS ((EXCL VARROA MITES)) - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, AFFECTED BY UNKNOWN CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, AFFECTED BY UNKNOWN CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, AFFECTED BY VARROA MITES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, AFFECTED BY VARROA MITES - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, RENOVATED - INVENTORY, MEASURED IN COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, RENOVATED - INVENTORY, MEASURED IN COLONIES - <b>CV (%)</b>","HONEY, BEE COLONIES, RENOVATED - INVENTORY, MEASURED IN PCT OF COLONIES - <b>VALUE</b>","HONEY, BEE COLONIES, RENOVATED - INVENTORY, MEASURED IN PCT OF COLONIES - <b>CV (%)</b>"
0,SURVEY,2023,APR THRU JUN,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,HONEY,TOTAL,NOT SPECIFIED,"596,360",NaN,NaN,NaN,"54,730",NaN,"237,350",NaN,9,NaN,6.5,NaN,13.6,NaN,10.5,NaN,13.9,NaN,6,NaN,50.9,NaN,"478,440",NaN,18,NaN
1,SURVEY,2023,APR THRU JUN,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,HONEY,TOTAL,NOT SPECIFIED,"4,200",NaN,"15,500",NaN,250,NaN,"2,000",NaN,13,NaN,2.1,NaN,4.4,NaN,1,NaN,10.4,NaN,9.1,NaN,37.1,NaN,"3,800",NaN,25,NaN
2,SURVEY,2023,APR THRU JUN,NaN,STATE,ARIZONA,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,HONEY,TOTAL,NOT SPECIFIED,"4,900",NaN,"34,000",NaN,30,NaN,"8,500",NaN,25,NaN,2.1,NaN,0.7,NaN,0,NaN,0,NaN,(Z),NaN,15.4,NaN,"2,400",NaN,7,NaN
3,SURVEY,2023,APR THRU JUN,NaN,STATE,ARKANSAS,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,HONEY,TOTAL,NOT SPECIFIED,"6,000",NaN,"24,000",NaN,"1,400",NaN,"1,900",NaN,8,NaN,2.8,NaN,1.9,NaN,7.4,NaN,13.1,NaN,3.1,NaN,22.4,NaN,"2,200",NaN,9,NaN
4,SURVEY,2023,APR THRU JUN,NaN,STATE,CALIFORNIA,6.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,HONEY,TOTAL,NOT SPECIFIED,"131,000",NaN,"930,000",NaN,"2,900",NaN,"52,000",NaN,6,NaN,4,NaN,11.3,NaN,4.5,NaN,3.3,NaN,4.1,NaN,41.6,NaN,"112,000",NaN,12,NaN


In [5]:
raw_df.notnull().sum()

Program                                                                                                                    1559
Year                                                                                                                       1559
Period                                                                                                                     1559
Week Ending                                                                                                                   0
Geo Level                                                                                                                  1559
State                                                                                                                      1559
State ANSI                                                                                                                 1481
Ag District                                                                                             

In [6]:
raw_df = raw_df.dropna(axis=1, how='all')

In [7]:
raw_df = raw_df[raw_df['Domain Category'] != 'INVENTORY OF BEE COLONIES: (LESS THAN 5 COLONIES)']

In [8]:
columns_to_remove = [col for col in raw_df.columns if 'CV' in col] + \
['Program', 'Geo Level', 'Domain Category', 'State ANSI', 'watershed_code', 'Commodity', 'Domain']
                                                                     
raw_df.drop(columns=columns_to_remove, inplace=True)

In [9]:
raw_df = raw_df.rename(columns={'HONEY, BEE COLONIES - ADDED & REPLACED, MEASURED IN COLONIES  -  <b>VALUE</b>' : 'Colonies_Added_And_Replaced',
                                'HONEY, BEE COLONIES - INVENTORY, MAX, MEASURED IN COLONIES  -  <b>VALUE</b>' : 'Max_Colonies',
                                'HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DISORDER, MEASURED IN COLONIES  -  <b>VALUE</b>' : 'Num_Affected_Colony_Collapse_Disorder',
                                'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN COLONIES  -  <b>VALUE</b>' : 'Num_Affected_Deadout',
                                'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Deadout',
                                'HONEY, BEE COLONIES, AFFECTED BY DISEASE - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Disease',
                                'HONEY, BEE COLONIES, AFFECTED BY OTHER CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Other',
                                'HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Pesticides',
                                'HONEY, BEE COLONIES, AFFECTED BY PESTS ((EXCL VARROA MITES)) - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Pests',
                                'HONEY, BEE COLONIES, AFFECTED BY UNKNOWN CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Unknown',
                                'HONEY, BEE COLONIES, AFFECTED BY VARROA MITES - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Affected_Varroa_Mites',
                                'HONEY, BEE COLONIES, RENOVATED - INVENTORY, MEASURED IN COLONIES  -  <b>VALUE</b>' : 'Num_Renovated',
                                'HONEY, BEE COLONIES, RENOVATED - INVENTORY, MEASURED IN PCT OF COLONIES  -  <b>VALUE</b>' : 'Pct_Renovated'
                               })
                               

In [10]:
raw_df = raw_df.replace(',', '', regex=True)

In [11]:
raw_df.iloc[:, 3:] = raw_df.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')

In [12]:
raw_df.head()

,Year,Period,State,Colonies_Added_And_Replaced,Max_Colonies,Num_Affected_Colony_Collapse_Disorder,Num_Affected_Deadout,Pct_Affected_Deadout,Pct_Affected_Disease,Pct_Affected_Other,Pct_Affected_Pesticides,Pct_Affected_Pests,Pct_Affected_Unknown,Pct_Affected_Varroa_Mites,Num_Renovated,Pct_Renovated
0,2023,APR THRU JUN,US TOTAL,596360.0,NaN,54730.0,237350.0,9.0,6.5,13.6,10.5,13.9,6.0,50.9,478440.0,18.0
1,2023,APR THRU JUN,ALABAMA,4200.0,15500.0,250.0,2000.0,13.0,2.1,4.4,1.0,10.4,9.1,37.1,3800.0,25.0
2,2023,APR THRU JUN,ARIZONA,4900.0,34000.0,30.0,8500.0,25.0,2.1,0.7,0.0,0.0,NaN,15.4,2400.0,7.0
3,2023,APR THRU JUN,ARKANSAS,6000.0,24000.0,1400.0,1900.0,8.0,2.8,1.9,7.4,13.1,3.1,22.4,2200.0,9.0
4,2023,APR THRU JUN,CALIFORNIA,131000.0,930000.0,2900.0,52000.0,6.0,4.0,11.3,4.5,3.3,4.1,41.6,112000.0,12.0


In [13]:
national_df = raw_df[(raw_df['State'] == 'US TOTAL')]
state_df = raw_df[(raw_df['State'] != 'US TOTAL')]

In [14]:
state_df = state_df.dropna(axis=1, how='all')
national_df = national_df.dropna(axis=1, how='all')

In [15]:
state_df['Period'] = state_df['Period'].map({
    'JAN THRU MAR': 'Q1',
    'APR THRU JUN': 'Q2',
    'JUL THRU SEP': 'Q3',
    'OCT THRU DEC': 'Q4',
})

national_df['Period'] = national_df['Period'].map({
    'JAN THRU MAR': 'Q1',
    'APR THRU JUN': 'Q2',
    'JUL THRU SEP': 'Q3',
    'OCT THRU DEC': 'Q4',
})

In [16]:
national_df['Quarter'] = national_df['Year'].astype(str) + '_' + national_df['Period']
state_df['Quarter'] = state_df['Year'].astype(str) + '_' + state_df['Period']

In [17]:
national_df = national_df.drop(columns=['Period', 'Year'])
state_df = state_df.drop(columns=['Period', 'Year'])

In [18]:
national_df = national_df[['Quarter'] + [col for col in national_df.columns if col != 'Quarter']]
state_df = state_df[['Quarter'] + [col for col in state_df.columns if col != 'Quarter']]

In [19]:
national_df = national_df.sort_values(by='Quarter', ascending=True)
state_df = state_df.sort_values(by='Quarter', ascending=True)

national_df = national_df.reset_index(drop=True)
state_df = state_df.reset_index(drop=True)  # Keep the state_df's index independent

In [20]:
state_df.head()

,Quarter,State,Colonies_Added_And_Replaced,Max_Colonies,Num_Affected_Colony_Collapse_Disorder,Num_Affected_Deadout,Pct_Affected_Deadout,Pct_Affected_Disease,Pct_Affected_Other,Pct_Affected_Pesticides,Pct_Affected_Pests,Pct_Affected_Unknown,Pct_Affected_Varroa_Mites,Num_Renovated,Pct_Renovated
0,2015_Q1,MISSOURI,220.0,13000.0,NaN,2000.0,15.0,2.2,11.8,4.2,6.0,6.3,13.1,910.0,7.0
1,2015_Q1,NEBRASKA,120.0,31000.0,NaN,2600.0,8.0,0.1,0.3,0.5,0.1,3.0,5.9,310.0,1.0
2,2015_Q1,NEW JERSEY,570.0,9000.0,NaN,1100.0,12.0,30.4,19.5,11.1,13.2,16.7,40.1,1600.0,18.0
3,2015_Q1,NEW MEXICO,190.0,7000.0,NaN,690.0,10.0,0.0,1.3,0.2,0.1,0.4,17.1,600.0,9.0
4,2015_Q1,NEW YORK,430.0,30000.0,NaN,6500.0,22.0,0.5,12.4,4.4,2.6,8.3,7.6,950.0,3.0


In [21]:
state_sum = state_df.groupby('Quarter')['Max_Colonies'].sum().reset_index()
national_df = national_df.merge(state_sum, on='Quarter', how='left')
national_df = national_df.rename(columns={'Max_Colonies_x': 'Max_Colonies'})

In [22]:
national_df.head()

,Quarter,State,Colonies_Added_And_Replaced,Num_Affected_Colony_Collapse_Disorder,Num_Affected_Deadout,Pct_Affected_Deadout,Pct_Affected_Disease,Pct_Affected_Other,Pct_Affected_Pesticides,Pct_Affected_Pests,Pct_Affected_Unknown,Pct_Affected_Varroa_Mites,Num_Renovated,Pct_Renovated,Max_Colonies
0,2015_Q1,US TOTAL,546980.0,92250.0,500020.0,18.0,3.1,6.9,7.4,8.6,4.3,25.2,270530.0,10.0,3619490.0
1,2015_Q2,US TOTAL,661860.0,56760.0,352860.0,12.0,4.9,11.6,16.6,19.5,3.5,43.4,692850.0,24.0,3909400.0
2,2015_Q3,US TOTAL,172990.0,88390.0,457100.0,15.0,8.0,8.8,15.2,17.6,4.8,41.2,303070.0,10.0,3535480.0
3,2015_Q4,US TOTAL,117150.0,87610.0,412380.0,14.0,5.2,7.3,9.5,11.4,6.9,37.0,158790.0,6.0,3717460.0
4,2016_Q1,US TOTAL,571880.0,115950.0,416100.0,16.0,6.2,6.9,10.9,12.6,5.4,34.6,245060.0,9.0,3309840.0


In [23]:
state_file_path = os.path.join(data_folder, "state_data.csv")
national_file_path = os.path.join(data_folder, "national_data.csv")

state_df.to_csv(state_file_path, index=False)
national_df.to_csv(national_file_path, index=False)

print("Data Saved")